In [2]:
import collections

import datasets
import numpy as np
import pandas as pd
import re
import rich
import rich.console
import rich.markup
import rich.table
import torch
import transformers

datasets.disable_caching()
CONSOLE = rich.console.Console(width=100, force_terminal=True)

def gsm8k_entry_viewer(entry):
    table = rich.table.Table(title="Entry", show_lines=True, title_justify="left")
    table.add_column("Key")
    table.add_column("Value")
    assert len(entry) == 2

    for key in ["question", "answer"]:
        text = rich.markup.escape(entry[key])
        text = text.replace("<<", "[bold green]<<[/]")
        text = text.replace(">>", "[bold green]>>[/]")
        text = text.replace("=", "[bold white on red]=[/]")
        table.add_row(f"{key.capitalize()}:", text)

    CONSOLE.print(table)

In [7]:
def count_tags(text):
    tag_count = 0
    state_started = False

    for i, c in enumerate(text):
        at_least_one_more_left = i < len(text) - 2
        if at_least_one_more_left:
            if not state_started and c == "<" and text[i + 1] == "<":
                state_started = True

            elif state_started and c == ">" and text[i + 1] == ">":
                state_started = False
                tag_count += 1

    return tag_count

def count_char(char, text):
    proposed = sum(c == char for c in text)
    return proposed

def count_equals(text):
    return count_char("=", text)

def build_annotated_object():
    gsm8k = datasets.load_dataset("gsm8k", "main", split="train")

    annotated = []

    # count the number of equations in each answer.
    for entry in gsm8k:
        answer = entry["answer"]
        tag_count = count_tags(answer)
        equals_count = count_equals(answer)
        annotated.append({"entry": entry, "tag_count": tag_count, "equals_count": equals_count})

    return pd.DataFrame(annotated)

def filter_gsm8k(annotated: pd.DataFrame):
    tag_counts = set(annotated["tag_count"].values)
    print(tag_counts)

    final_filtered = []
    for tag_count in tag_counts:
        assert isinstance(tag_count, (int, np.int64)), type(tag_count).mro()
        values: pd.DataFrame = annotated[annotated["tag_count"] == tag_count]
        filter_ = values["equals_count"] == 2 * tag_count
        num_good = filter_.sum()
        filtered: pd.DataFrame = values[filter_]
        final_filtered.append(filtered)
        print(f"Tag count {tag_count} has {len(values)} entries.")
        print(f"Tag count {tag_count} has {num_good} entries with the right number of equals signs.")
        print("--")
    
    # Merge the dataframes together.
    final_filtered = pd.concat(final_filtered)
    print(f"Filtered to {len(final_filtered)} entries, which is {len(final_filtered) / len(annotated):.0%} of the dataset.")
    
    return final_filtered

annotated = build_annotated_object()
filtered = filter_gsm8k(annotated)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Tag count 0 has 95 entries.
Tag count 0 has 2 entries with the right number of equals signs.
--
Tag count 1 has 404 entries.
Tag count 1 has 16 entries with the right number of equals signs.
--
Tag count 2 has 2175 entries.
Tag count 2 has 1795 entries with the right number of equals signs.
--
Tag count 3 has 2137 entries.
Tag count 3 has 1810 entries with the right number of equals signs.
--
Tag count 4 has 1424 entries.
Tag count 4 has 1243 entries with the right number of equals signs.
--
Tag count 5 has 785 entries.
Tag count 5 has 693 entries with the right number of equals signs.
--
Tag count 6 has 287 entries.
Tag count 6 has 259 entries with the right number of equals signs.
--
Tag count 7 has 123 entries.
Tag count 7 has 104 entries with the right number of equals signs.
--
Tag count 8 has 40 entries.
Tag count 8 has 36 entries with the right number of equals signs.
--
Tag count 9 has 3 entries.
Tag count 9 has 3 entries with the right number of 

In [ ]:
TAG_COUNT = 1
EQUALS_COUNT = 3

base_filter = annotated.tag_count == TAG_COUNT
base_entries = annotated[base_filter]
len_base = len(base_entries)
joint = pd.concat({"Counts": base_entries.value_counts("equals_count"), "Percentages": (base_entries.value_counts("equals_count") / len_base)}, axis=1).sort_index()
print(joint.to_string(float_format=r"{:.1%}".format))

# index = (base_entries.value_counts("equals_count") / len_base).sort_index()
# CONSOLE.print(index.to_string(float_format=r"{:.1%}".format))
# eoi = annotated[base_filter & (annotated.equals_count == EQUALS_COUNT)]
# len_eoi = len(eoi)
# print(f"{len_eoi}/{len_base}, {len_eoi/len_base:.1%}")

# for _, row in eoi.head(5).iterrows():
#     gsm8k_entry_viewer(row['entry'])


In [8]:
ds_socratic = datasets.load_dataset("gsm8k", "socratic", split="train")
ds_regular = datasets.load_dataset("gsm8k", "main", split="train")

In [14]:
for i in range(15):
    socratic = ds_socratic["answer"][i]
    regular = ds_regular["answer"][i]
    
    num_tags = count_tags(regular)
    num_equals = count_equals(regular)
    num_lines_socratic = count_char("\n", socratic)
    num_lines_regular = count_char("\n", regular)
    hypothesis = num_equals == (num_tags + num_lines_regular)

    table = rich.table.Table(title="Entry", show_lines=True, title_justify="left")
    
    table.add_column(  "Key")
    table.add_column("Value")

    color_socratic   = "[bold white on red]"  if num_lines_socratic != num_tags else ""
    color_regular    = "[bold white on red]"  if num_lines_regular  != num_tags else ""
    color_num_equals = "[bold white on blue]" if num_equals         != 2 * num_tags else "" 
    color_hypothesis = "[bold white on purple]" if not hypothesis else ""

    table.add_row(                             "Num tags:", str(          num_tags))
    table.add_row(color_socratic   +  "Num lines regular:", str( num_lines_regular))
    table.add_row(color_regular    + "Num lines socratic:", str(num_lines_socratic))
    table.add_row(color_num_equals +         "Num equals:", str(        num_equals))
    table.add_row(                              "Regular:", regular .replace("\n", "\n------\n"))
    table.add_row(                             "Socratic:", socratic.replace("\n", "\n------\n"))
    table.add_row(color_hypothesis +         "Hypothesis:", str(hypothesis))

    CONSOLE.print(table)

Entry                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                 ┃ Value                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Num tags:           │ 2                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines regular:  │ 2                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines socratic: │ 2                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num equals:         │ 4                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Regular:            │ Natalia sold 48/2 = <<48/2=24>>24 clips in May.                            │
│                     │ ------                                                                     │
│                     │ Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.     │
│                     │ ------                                                                     │
│                     │ #### 72                                                                    │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Socratic:           │ How many clips did Natalia sell in May? ** Natalia sold 48/2 =             │
│                     │ <<48/2=24>>24 clips in May.                                                │
│                     │ ------                                                                     │
│                     │ How many clips did Natalia sell altogether in April and May? ** Natalia    │
│                     │ sold 48+24 = <<48+24=72>>72 clips altogether in April and May.             │
│                     │ ------                                                                     │
│                     │ #### 72                                                                    │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Hypothesis:         │ True                                                                       │
└─────────────────────┴────────────────────────────────────────────────────────────────────────────┘

Entry                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                 ┃ Value                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Num tags:           │ 2                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines regular:  │ 2                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines socratic: │ 2                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num equals:         │ 4                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Regular:            │ Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.                           │
│                     │ ------                                                                     │
│                     │ Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.                │
│                     │ ------                                                                     │
│                     │ #### 10                                                                    │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Socratic:           │ How much does Weng earn per minute? ** Weng earns 12/60 =                  │
│                     │ $<<12/60=0.2>>0.2 per minute.                                              │
│                     │ ------                                                                     │
│                     │ How much did Weng earn? ** Working 50 minutes, she earned 0.2 x 50 =       │
│                     │ $<<0.2*50=10>>10.                                                          │
│                     │ ------                                                                     │
│                     │ #### 10                                                                    │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Hypothesis:         │ True                                                                       │
└─────────────────────┴────────────────────────────────────────────────────────────────────────────┘

Entry                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                 ┃ Value                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Num tags:           │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines regular:  │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines socratic: │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num equals:         │ 6                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Regular:            │ In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.                │
│                     │ ------                                                                     │
│                     │ Betty's grandparents gave her 15 * 2 = $<<15*2=30>>30.                     │
│                     │ ------                                                                     │
│                     │ This means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.    │
│                     │ ------                                                                     │
│                     │ #### 5                                                                     │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Socratic:           │ How much money does Betty have in the beginning? ** In the beginning,      │
│                     │ Betty has only 100 / 2 = $<<100/2=50>>50.                                  │
│                     │ ------                                                                     │
│                     │ How much money did Betty's grandparents give her? ** Betty's grandparents  │
│                     │ gave her 15 * 2 = $<<15*2=30>>30.                                          │
│                     │ ------                                                                     │
│                     │ How much more money does Betty need to buy the wallet? ** This means,      │
│                     │ Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.                │
│                     │ ------                                                                     │
│                     │ #### 5                                                                     │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Hypothesis:         │ True                                                                       │
└─────────────────────┴────────────────────────────────────────────────────────────────────────────┘

Entry                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                 ┃ Value                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Num tags:           │ 4                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines regular:  │ 4                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines socratic: │ 4                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num equals:         │ 8                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Regular:            │ Maila read 12 x 2 = <<12*2=24>>24 pages today.                             │
│                     │ ------                                                                     │
│                     │ So she was able to read a total of 12 + 24 = <<12+24=36>>36 pages since    │
│                     │ yesterday.                                                                 │
│                     │ ------                                                                     │
│                     │ There are 120 - 36 = <<120-36=84>>84 pages left to be read.                │
│                     │ ------                                                                     │
│                     │ Since she wants to read half of the remaining pages tomorrow, then she     │
│                     │ should read 84/2 = <<84/2=42>>42 pages.                                    │
│                     │ ------                                                                     │
│                     │ #### 42                                                                    │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Socratic:           │ How many pages did Maila read today? ** Maila read 12 x 2 = <<12*2=24>>24  │
│                     │ pages today.                                                               │
│                     │ ------                                                                     │
│                     │ How many pages did Maila read since yesterday? ** So she was able to read  │
│                     │ a total of 12 + 24 = <<12+24=36>>36 pages since yesterday.                 │
│                     │ ------                                                                     │
│                     │ How many pages are left to be read? ** There are 120 - 36 =                │
│                     │ <<120-36=84>>84 pages left to be read.                                     │
│                     │ ------                                                                     │
│                     │ How many pages should she read tomorrow? ** Since she wants to read half   │
│                     │ of the remaining pages tomorrow, then she should read 84/2 = <<84/2=42>>42 │
│                     │ pages.                                                                     │
│                     │ ------                                                                     │
│                     │ #### 42                                                                    │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Hypothesis:         │ True                                 

Entry                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                 ┃ Value                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Num tags:           │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines regular:  │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines socratic: │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num equals:         │ 6                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Regular:            │ He writes each friend 3*2=<<3*2=6>>6 pages a week                          │
│                     │ ------                                                                     │
│                     │ So he writes 6*2=<<6*2=12>>12 pages every week                             │
│                     │ ------                                                                     │
│                     │ That means he writes 12*52=<<12*52=624>>624 pages a year                   │
│                     │ ------                                                                     │
│                     │ #### 624                                                                   │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Socratic:           │ How many pages does he write each week? ** He writes each friend           │
│                     │ 3*2=<<3*2=6>>6 pages a week                                                │
│                     │ ------                                                                     │
│                     │ How many pages does he write every week? ** So he writes 6*2=<<6*2=12>>12  │
│                     │ pages every week                                                           │
│                     │ ------                                                                     │
│                     │ How many pages does he write a year? ** That means he writes               │
│                     │ 12*52=<<12*52=624>>624 pages a year                                        │
│                     │ ------                                                                     │
│                     │ #### 624                                                                   │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Hypothesis:         │ True                                                                       │
└─────────────────────┴────────────────────────────────────────────────────────────────────────────┘

...

Entry                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                 ┃ Value                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Num tags:           │ 2                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines regular:  │ 5                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines socratic: │ 5                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num equals:         │ 7                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Regular:            │ Let S be the number of people on the first hundred years’ ship.            │
│                     │ ------                                                                     │
│                     │ The second hundred years’ ship had twice as many as the first, so it had   │
│                     │ 2S people.                                                                 │
│                     │ ------                                                                     │
│                     │ The third hundred years’ ship had twice as many as the second, so it had 2 │
│                     │ * 2S = <<2*2=4>>4S people.                                                 │
│                     │ ------                                                                     │
│                     │ All the ships had S + 2S + 4S = 7S = 847 people.                           │
│                     │ ------                                                                     │
│                     │ Thus, the ship that the monster ate in the first hundred years had S = 847 │
│                     │ / 7 = <<847/7=121>>121 people on it.                                       │
│                     │ ------                                                                     │
│                     │ #### 121                                                                   │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Socratic:           │ Define a variable ** Let S be the number of people on the first hundred    │
│                     │ years’ ship.                                                               │
│                     │ ------                                                                     │
│                     │ How many people were on the second hundred years’ ship? ** The second      │
│                     │ hundred years’ ship had twice as many as the first, so it had 2S people.   │
│                     │ ------                                                                     │
│                     │ How many people were on the third hundred years’ ship? ** The third        │
│                     │ hundred years’ ship had twice as many as the second, so it had 2 * 2S =    │
│                     │ <<2*2=4>>4S people.                                                        │
│                     │ ------                                                                     │
│                     │ How many people were on the ships? ** All the ships had S + 2S + 4S = 7S = │
│                     │ 847 people.                                                                │
│                     │ ------                               

Entry                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                 ┃ Value                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Num tags:           │ 5                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines regular:  │ 5                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines socratic: │ 5                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num equals:         │ 10                                                                         │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Regular:            │ He saved up $110 total because 95 + 15 = <<95+15=110>>110                  │
│                     │ ------                                                                     │
│                     │ He saved $15 from his allowance because 3 x 5 = <<3*5=15>>15               │
│                     │ ------                                                                     │
│                     │ He earned $60 mowing lawns because 4 x 15 = <<4*15=60>>60                  │
│                     │ ------                                                                     │
│                     │ He earned $35 shoveling driveways because 110 - 60 - 15 =                  │
│                     │ <<110-60-15=35>>35                                                         │
│                     │ ------                                                                     │
│                     │ He shoveled 5 driveways because 35 / 7 = <<35/7=5>>5                       │
│                     │ ------                                                                     │
│                     │ #### 5                                                                     │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Socratic:           │ How much money did Tobias save up? ** He saved up $110 total because 95 +  │
│                     │ 15 = <<95+15=110>>110                                                      │
│                     │ ------                                                                     │
│                     │ How much money did Tobias save from his allowance? ** He saved $15 from    │
│                     │ his allowance because 3 x 5 = <<3*5=15>>15                                 │
│                     │ ------                                                                     │
│                     │ How much money did Tobias earn mowing lawns? ** He earned $60 mowing lawns │
│                     │ because 4 x 15 = <<4*15=60>>60                                             │
│                     │ ------                                                                     │
│                     │ How much money did Tobias earn shoveling driveways? ** He earned $35       │
│                     │ shoveling driveways because 110 - 60 - 15 = <<110-60-15=35>>35             │
│                     │ ------                                                                     │
│                     │ How many driveways did Tobias shovel? ** He shoveled 5 driveways because   │
│                     │ 35 / 7 = <<35/7=5>>5                                                       │
│                     │ ------                               

Entry                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                 ┃ Value                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Num tags:           │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines regular:  │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines socratic: │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num equals:         │ 6                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Regular:            │ Half of the number of Randy's mango trees is 60/2 = <<60/2=30>>30 trees.   │
│                     │ ------                                                                     │
│                     │ So Randy has 30 - 5 = <<30-5=25>>25 coconut trees.                         │
│                     │ ------                                                                     │
│                     │ Therefore, Randy has 60 + 25 = <<60+25=85>>85 treeson his farm.            │
│                     │ ------                                                                     │
│                     │ #### 85                                                                    │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Socratic:           │ How many mango trees does Randy have? ** Half of the number of Randy's     │
│                     │ mango trees is 60/2 = <<60/2=30>>30 trees.                                 │
│                     │ ------                                                                     │
│                     │ How many coconut trees does Randy have? ** So Randy has 30 - 5 =           │
│                     │ <<30-5=25>>25 coconut trees.                                               │
│                     │ ------                                                                     │
│                     │ How many trees does Randy have in all on his farm? ** Therefore, Randy has │
│                     │ 60 + 25 = <<60+25=85>>85 treeson his farm.                                 │
│                     │ ------                                                                     │
│                     │ #### 85                                                                    │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Hypothesis:         │ True                                                                       │
└─────────────────────┴────────────────────────────────────────────────────────────────────────────┘

Entry                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                 ┃ Value                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Num tags:           │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines regular:  │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines socratic: │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num equals:         │ 6                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Regular:            │ A pound of cream cheese cost $10 / 2 = $<<10/2=5>>5.                       │
│                     │ ------                                                                     │
│                     │ A pack of cold cuts cost $10 x 2 = $<<10*2=20>>20.                         │
│                     │ ------                                                                     │
│                     │ Jasper spent $10 + $5 + $20 = $<<10+5+20=35>>35 on the ingredients.        │
│                     │ ------                                                                     │
│                     │ #### 35                                                                    │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Socratic:           │ How much does a pound of cream cheese cost? ** A pound of cream cheese     │
│                     │ cost $10 / 2 = $<<10/2=5>>5.                                               │
│                     │ ------                                                                     │
│                     │ How much does a pack of cold cuts cost? ** A pack of cold cuts cost $10 x  │
│                     │ 2 = $<<10*2=20>>20.                                                        │
│                     │ ------                                                                     │
│                     │ How much did Jasper spend on the ingredients? ** Jasper spent $10 + $5 +   │
│                     │ $20 = $<<10+5+20=35>>35 on the ingredients.                                │
│                     │ ------                                                                     │
│                     │ #### 35                                                                    │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Hypothesis:         │ True                                                                       │
└─────────────────────┴────────────────────────────────────────────────────────────────────────────┘

Entry                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                 ┃ Value                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Num tags:           │ 2                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines regular:  │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num lines socratic: │ 3                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Num equals:         │ 4                                                                          │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Regular:            │ In one hour, there are 3 sets of 20 minutes.                               │
│                     │ ------                                                                     │
│                     │ So, Joy can read 8 x 3 = <<8*3=24>>24 pages in an hour.                    │
│                     │ ------                                                                     │
│                     │ It will take her 120/24 = <<120/24=5>>5 hours to read 120 pages.           │
│                     │ ------                                                                     │
│                     │ #### 5                                                                     │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Socratic:           │ How many minutes are in an hour? ** In one hour, there are 3 sets of 20    │
│                     │ minutes.                                                                   │
│                     │ ------                                                                     │
│                     │ How many pages can Joy read in an hour? ** So, Joy can read 8 x 3 =        │
│                     │ <<8*3=24>>24 pages in an hour.                                             │
│                     │ ------                                                                     │
│                     │ How many hours will it take Joy to read 120 pages? ** It will take her     │
│                     │ 120/24 = <<120/24=5>>5 hours to read 120 pages.                            │
│                     │ ------                                                                     │
│                     │ #### 5                                                                     │
├─────────────────────┼────────────────────────────────────────────────────────────────────────────┤
│ Hypothesis:         │ False                                                                      │
└─────────────────────┴────────────────────────────────────────────────────────────────────────────┘